In [2]:
#cifer or imagenet
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# train split to validation
from sklearn.model_selection import train_test_split
x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.2)

x_train1 = x_train1.astype('float32') / 255 
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

from keras.utils.np_utils import to_categorical
N = 10
y_train1 = to_categorical(y_train1, N)#.reshape(-1,1,1,10)
y_valid = to_categorical(y_valid, N)#.reshape(-1,1,1,10)
y_test = to_categorical(y_test, N)#.reshape(-1,1,1,10)

print(x_train1.shape)
print(x_valid.shape)
print(x_test.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, SpatialDropout2D, BatchNormalization, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.callbacks import ModelCheckpoint

def Model():
  
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size=(5,5), strides = 1, padding='same', input_shape=(32, 32, 3)))
    #model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),  strides = 2))
    
    model.add(Conv2D(192, kernel_size=(5,5), strides = 1, padding='same'))
    #model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),  strides = 2))
    
    model.add(Conv2D(384, kernel_size=(3,3), strides = 1, padding='same'))
    #model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),  strides = 2))
    
    model.add(Conv2D(256, kernel_size=(5,5), strides = 1, padding='same'))
    #model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),  strides = 2))
    
    model.add(Conv2D(256, kernel_size=(5,5), strides = 1, padding='same'))
    #model.add(BatchNormalization())
    model.add(SpatialDropout2D(rate=0.2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
    
    #model.add(Activation('relu'))
    model.add(Flatten())
    
    #model.add(Conv2D(4096, kernel_size=(1,1), strides = 1, padding='same'))
    model.add(Dense(4096))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.2))
    
    #model.add(Conv2D(4096, kernel_size=(1,1), strides = 1, padding='same'))
    model.add(Dense(4096))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.2))
    
    #model.add(Conv2D(10, kernel_size=(1,1), strides = 1, padding='same'))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    return model

model = Model()

from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              # optimizer=RMSprop(learning_rate = 0.001),
              optimizer = 'adam',
              metrics=['accuracy'])
checkpoint_cb = ModelCheckpoint("Model_50_55355_32_0.2_0.2_adam", save_best_only=True)

In [4]:
# verbose = progress bar setting
history = model.fit(x_train1, y_train1, batch_size=32, epochs=50, verbose=1, validation_data=(x_valid, y_valid), callbacks=[checkpoint_cb])

Epoch 1/50
1250/1250 [==============================] - 69s 19ms/step - loss: 2.1695 - accuracy: 0.1598 - val_loss: 1.6688 - val_accuracy: 0.3711
INFO:tensorflow:Assets written to: Model_50_55355_32_0.2_0.2_adam/assets
Epoch 2/50
1250/1250 [==============================] - 24s 20ms/step - loss: 1.7191 - accuracy: 0.3591 - val_loss: 1.4971 - val_accuracy: 0.4571
INFO:tensorflow:Assets written to: Model_50_55355_32_0.2_0.2_adam/assets
Epoch 3/50
1250/1250 [==============================] - 24s 19ms/step - loss: 1.5671 - accuracy: 0.4251 - val_loss: 1.3590 - val_accuracy: 0.5065
INFO:tensorflow:Assets written to: Model_50_55355_32_0.2_0.2_adam/assets
Epoch 4/50
1250/1250 [==============================] - 25s 20ms/step - loss: 1.4763 - accuracy: 0.4622 - val_loss: 1.3212 - val_accuracy: 0.5303
INFO:tensorflow:Assets written to: Model_50_55355_32_0.2_0.2_adam/assets
Epoch 5/50
1250/1250 [==============================] - 23s 19ms/step - loss: 1.4011 - accuracy: 0.5005 - val_loss: 1.2089 -

In [5]:
from keras.models import load_model
import numpy as np
# model.save_weights("mnist.hdf5") 
model = load_model("Model_50_55355_32_0.2_0.2_adam")

score = model.evaluate(x_train1, y_train1, verbose=0)
print('Training loss:', score[0])
print('Training accuracy:', score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Training loss: 0.42032623291015625
Training accuracy: 0.8754749894142151
Test loss: 0.8167001008987427
Test accuracy: 0.734499990940094


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
np.save("Model_50_55355_32_0.2_0.2_adam",history.history)
# plot
plt.plot(history.history['accuracy'], marker='.', label='acc')
plt.plot(history.history['val_accuracy'], marker='.', label='val_acc')
plt.plot(history.history['score[1]'], marker='.', label='val_acc')

plt.title('Model accuracy')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.show()

# plot
plt.plot(history.history['loss'], marker='.', label='loss')
plt.plot(history.history['val_loss'], marker='.', label='val_loss')
plt.title('Model loss')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc='best')
plt.show()

NameError: ignored

# New Section